### Group By Having and Count

Now that you can select raw data, you're ready to learn **how to group** your data and **count things** within those groups.

This can help you answer questions like:

- How many of each kind of fruit has our store sold?
- How many species of animal has the vet office treated?

##### COUNT
- Returns a count of things. If you pass it the name of a column, it will return the number of entries in that column.
- COUNT() is an example of an aggregate function, which takes many values and returns one.

##### GROUP BY
- It takes the name of one or more columns, and **treats all rows with the same value in that column as a single group** when you apply aggregate functions


##### GROUP BY ... HAVING
- HAVING is used in combination with GROUP BY to ignore groups that don't meet certain criteria.

Here is an example. Suppose this table:

|  ID   |        Name        | Animal |
| :---: | :----------------: | :----: |
|   1   | Dr. Harris Bonkers | Rabbit |
|   2   |        Moon        |  Dog   |
|   3   |       Ripley       |  Cat   |
|   4   |        Tom         |  Cat   |

In [1]:
# First, you can SELECT directly from a COUNT command:
# It means that COUNT adds a new colum
QUERY = """
    SELECT COUNT(ID)
    FROM 'bigquery-public-data.pet_records.pets'
"""
# It's going to return something like this
response = {
    'f0_': 4
}

In [2]:
# And here is an example with all techniques together

# For example, say we want to know how many of each type of animal
# we have in the pets table.

# We can use GROUP BY to group together rows that have the same
# value in the Animal column, while using COUNT() to find out
# how many ID's we have in each group with value > 1 using HAVING.

QUERY = """
    SELECT Animal COUNT(ID)
    FROM 'bigquery-public-data.pet_records.pets'
    GROUP BY Animal
    HAVING COUNT(ID) > 1
"""
# Other animals have just 1 records.
response = {
    'Animal': 'Cat',
    'f0_': 2
}

##### Important ⚠️

Let's consider this two queries.

- The first has two variables: **parent** and **id**
- The second has three variables: **author**, **parent** and **id**

In [3]:
query_good = """
    SELECT parent, COUNT(id)
    FROM `bigquery-public-data.hacker_news.comments`
    GROUP BY parent
"""
query_bad = """
    SELECT author, parent, COUNT(id)
    FROM `bigquery-public-data.hacker_news.comments`
    GROUP BY parent
"""

The second one is bad because **author** is not beign assigned either GROUP BY or COUNT commands.

From kaggle https://www.kaggle.com/dansbecker/group-by-having-count#Note-on-using-GROUP-BY

Note on using **GROUP BY**

Note that because it tells SQL how to apply aggregate functions, it doesn't make sense to use GROUP BY without an aggregate function. Similarly, if you have any GROUP BY clause, then all variables must be passed to either a

- **GROUP BY** command, or
- an **aggregation** function.

##### Example with real dataset

In [7]:
from google.cloud import bigquery
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file("secrets.json")
client = bigquery.Client(credentials=credentials)
dataset = client.get_dataset('bigquery-public-data.hacker_news')
table_ref = dataset.table("comments")
table = client.get_table(table_ref)
df = client.list_rows(table, max_results=5).to_dataframe()
df

,id,by,author,time,time_ts,text,parent,deleted,dead,ranking
0,2701393,5l,5l,1309184881,2011-06-27 14:28:01+00:00,And the glazier who fixed all the broken windo...,2701243,None,None,0
1,5811403,99,99,1370234048,2013-06-03 04:34:08+00:00,Does canada have the equivalent of H1B/Green c...,5804452,None,None,0
2,21623,AF,AF,1178992400,2007-05-12 17:53:20+00:00,"Speaking of Rails, there are other options in ...",21611,None,None,0
3,10159727,EA,EA,1441206574,2015-09-02 15:09:34+00:00,Humans and large livestock (and maybe even pet...,10159396,None,None,0
4,2988424,Iv,Iv,1315853580,2011-09-12 18:53:00+00:00,I must say I reacted in the same way when I re...,2988179,None,None,0


In [10]:
# Query to select comments that received more than 10 replies

# we can GROUP BY the parent column and COUNT() the id column in
# order to figure out the number of comments that were made as
# responses to a specific comment
query_popular = """
    SELECT parent, COUNT(id)
    FROM `bigquery-public-data.hacker_news.comments`
    GROUP BY parent
    HAVING COUNT(id) > 10
"""
query_job = client.query(query_popular)
popular_comments = query_job.to_dataframe()
popular_comments

,parent,f0_
0,4332978,53
1,2970550,63
2,3353593,68
3,3734303,56
4,5048699,61
...,...,...
77363,6917194,37
77364,9150163,37
77365,4917828,37
77366,8822723,37


Each row in the popular_comments DataFrame corresponds to a comment that received more than ten replies. For instance, the comment with ID 3734303 received 56 replies.
##### Improvements
- The column resulting from COUNT(id) was called f0__. That's not a very descriptive name. You can change the name by adding AS NumPosts after you specify the aggregation.
- If you are ever unsure what to put inside the COUNT() function, you can do COUNT(1) to count the rows in each group.